__Задача__

Используя данные о хоккеистах, проверьте, является ли среди хоккеистов из Финляндии, Норвегии и Дании значимым отличие: а) роста, б) BMI.

__Решение__

Гражданство игроков является категориальным признаком. Указанные гипотезы можно проверить с помощью однофакторного дисперсионного анализа. Для начала загрузим датасет:

In [2]:
import numpy as np
import pandas as pd

In [9]:
df = pd.read_csv('hockey_players.csv', encoding= 'unicode_escape')

df.head()


,year,country,no,name,position,side,height,weight,birth,club,age,cohort,bmi
0,2001,RUS,10,tverdovsky oleg,D,L,185,84.0,1976-05-18,anaheim mighty ducks,24.952772,1976,24.543462
1,2001,RUS,2,vichnevsky vitali,D,L,188,86.0,1980-03-18,anaheim mighty ducks,21.119781,1980,24.332277
2,2001,RUS,26,petrochinin evgeni,D,L,182,95.0,1976-02-07,severstal cherepovetal,25.229295,1976,28.680111
3,2001,RUS,28,zhdan alexander,D,R,178,85.0,1971-08-28,ak bars kazan,29.675565,1971,26.827421
4,2001,RUS,32,orekhovsky oleg,D,R,175,88.0,1977-11-03,dynamo moscow,23.490760,1977,28.734694


In [7]:
from google.colab import files
uploaded = files.upload()

Saving hockey_players.csv to hockey_players.csv


In [10]:
df['country'].unique()

array(['RUS', 'AUT', 'BLR', 'CAN', 'CZE', 'FIN', 'GER', 'ITA', 'JPN',
       'LAT', 'NOR', 'SUI', 'SVK', 'SWE', 'UKR', 'USA', 'POL', 'SLO',
       'DEN', 'FRA', 'KAZ', 'HUN'], dtype=object)

In [11]:
countries = ['FIN', 'NOR', 'DEN']

Теперь реализуем функцию для вычисления F-статистики:

In [13]:
def f_stat(*ys: np.ndarray) -> float:
    """Функция возвращает значение F-статистики
    для дисперсионного анализа.
    """
    y = np.concatenate(ys)

    ss_b = sum((yi.mean() - y.mean()) ** 2 * yi.size for yi in ys)
    ss_w = sum(((yi - yi.mean()) ** 2).sum() for yi in ys)
    
    sigma_b = ss_b / (len(ys) - 1)
    sigma_w = ss_w / (y.size - len(ys))
    
    return sigma_b / sigma_w

Создадим выборки роста хоккеистов для каждой из стран и посчитаем по ним F-статистику:

In [15]:
ys = [df.loc[df['country'] == country, 'height'] for country in countries]
F = f_stat(*ys)
F

4.273207343917213

Для построения критической области нам нужны квантили распределения Фишера с параметрами $k_1 = k - 1$, $k_2 = n - k$, где $k$ — число уровней, $n$ — общее число элементов во всех выборках. Примем уровень статистической значимости alpha = 0,05.

In [16]:
k = len(countries)
n = sum(y.size for y in ys)
k1 = k - 1
k2 = n - k

k1, k2


(2, 1023)

In [18]:
from scipy import stats

In [19]:
alpha = 0.05
t = stats.f.ppf(1 - alpha, k1, k2)
t


3.0045220661840073

Значит, критическая область: $$\Omega_\alpha = \left( 3.004, \infty \right)$$

Поскольку статистика попала в критическую область, заключаем, что между ростами хоккеистов выбранных стран действительно есть значимое отличие.

Аналогично для BMI. Пересчитать нужно только F-статистику, поскольку остальные параметры такие же, как раньше.

In [27]:
ys = [df.loc[df['country'] == country, 'bmi'] for country in countries]
F = f_stat(*ys)
F


36.05816727743814

Данное значение также попадает в критическую оюласть, поэтому для BMI гражданство игроков является значимым.